In [ ]:
%load_ext autoreload
%autoreload 2
from IPython import get_ipython; ipython = get_ipython()
#%run -i ../exec_HPC.py 0
import os 
import sys
from os.path import join as pjoin
p = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'state-space-adaptation' )
sys.path.append(p)
sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import pandas as pd
import time
import numpy as np
from config2 import path_data
import matplotlib.pyplot as plt

import state_space_Tan2014 as tan
from state_space_Tan2014 import *
import matplotlib.pyplot as plt

#df = pd.read_pickle('/home/demitau/ownCloud/Current/output/memerr/workstation_CRNL/row=trial.pkl.zip')

d = '/home/demitau/data_Quentin/full_experiments/context_change_behav/'
#fnb = 'dima_test_context_change_20230309_164851'
#fnb = '2023-SE1-pilot2_context_change_20230310_161928'
fnb = '2023-SE1-017_context_change_20230425_161049'
fn = fnb + '.log'
fnp = fnb + '.param'
fnf_dfcc = pjoin(d,'row=trial.pkl.zip')
dfcc_all     = pd.read_pickle(fnf_dfcc)      
dfcpc_all    = pd.read_pickle(pjoin(d,'row=pause.pkl.zip'))

scripts_dir = pjoin(p, 'state_space_Tan2014')
scr = pjoin(scripts_dir, 'run_example.py')

assert not dfcc_all.duplicated(['subject','trial_index']).any()
subjects = dfcc_all['subject'].unique()


fnf_NIH = pjoin(path_data,'df_all_multi_tsz.pkl.zip')
import datetime
print('NIH dataset mtime = ', datetime.datetime.fromtimestamp(os.path.getmtime(fnf_NIH)) )
dfcc_all_NIH_ = pd.read_pickle(fnf_NIH)
dfcc_all_NIH = dfcc_all_NIH_.query('trial_shift_size == 1 and trial_group_col_calc == "trials"').copy()
assert not dfcc_all_NIH.duplicated(['subject','trials']).any()

subjects_NIH = dfcc_all_NIH['subject'].unique()

# # main run
# nt = 1000
# nsub = 4
# subjects_sub = subjects[0:nsub]
# numtrain = 12
# dfc = df.query('subject in @subjects_sub and trial_index < @nt and trial_index >= @numtrain').copy()
# dfc.loc[dfc.query('error_lh2_ang_deg.abs() > 60').index, 'error_lh2_ang_deg'] = np.nan
# # TODO: maybe I should invalidate too big errors, they are clearly outliers
n_jobs = 8
numtrain = 12
# do_plot = 0
# #ipython.run_line_magic('run',' -i ' + scr)

#from taumodels.models import fitTanModel
inds = dfcc_all_NIH.query('environment == 1').index
dfcc_all_NIH.loc[inds,'perturbation'] = dfcc_all_NIH.loc[inds,'feedback'] - dfcc_all_NIH.loc[inds,'org_feedback']
dfcc_all_NIH.loc[inds,'perturbation'] *= 180 / np.pi
#[]
dfcc_all_NIH['trial_index'] = dfcc_all_NIH['trials']

In [ ]:
datetime.datetime.fromtimestamp(os.path.getmtime(fnf_NIH)).month

In [ ]:
dfcc_all_NIH.groupby('subject')['trial_index'].diff().min()

In [ ]:
from behav_proc import addWindowCols
dfcc_all_NIH = addWindowCols(dfcc_all_NIH)

def f(df):
    df['error_prod'] = df['error'] * df['error'].shift(1)
    return df
dfcc_all_NIH = dfcc_all_NIH.groupby(['subject'],group_keys=False).apply(f)
dfcc_all_NIH['prev_error_prod'] = dfcc_all_NIH.groupby(['subject'])['error_prod'].shift(1)


In [ ]:
dfcc_all_NIH['err_sens']

In [ ]:
coln_error = 'error_lh2_ang_deg'

In [ ]:
path_data_cc = '/home/demitau/data_Quentin/full_experiments/context_change_behav/results'

In [ ]:
fnf = pjoin(path_data_cc,'dfnamesubj2res_Tan.npz')
dfnamesubj2res_Tan = np.load(fnf, allow_pickle=1)
dfnamesubj2res_Tan = dfnamesubj2res_Tan['arr_0'][()]
print(len(dfnamesubj2res_Tan))

In [ ]:
fnf = pjoin(path_data_cc,'dfnamesubj2res_Died.npz')
dfnamesubj2res_Died = np.load(fnf, allow_pickle=1)
dfnamesubj2res_Died = dfnamesubj2res_Died['arr_0'][()]

In [ ]:
fnf = pjoin(path_data_cc,'dfnamesubj2res_Died_fboffl.npz')
dfnamesubj2res_Died_fboffl = np.load(fnf, allow_pickle=1)
dfnamesubj2res_Died_fboffl = dfnamesubj2res_Died_fboffl['arr_0'][()]

In [ ]:
fnf = pjoin(path_data_cc,'dfnamesubj2res_Herz.npz')
dfnamesubj2res_Herz = np.load(fnf, allow_pickle=1)
dfnamesubj2res_Herz = dfnamesubj2res_Herz['arr_0'][()]

In [ ]:
def getTanModelNte(s):
    p = dfnamesubj2res_Tan[('NIH',s)]['params']
    return p['NbPreviousTrials'], p['c']
ntes, cs = [getTanModelNte(s)[0] for s in subjects_NIH], [getTanModelNte(s)[1] for s in subjects_NIH]

In [ ]:
#getTanModelNte(subjects_NIH[5])

In [ ]:
dfmc = pd.DataFrame({'Nt':ntes,'C':cs})
dfmc.describe().T

In [ ]:
np.mean(  ntes), np.std(  ntes), np.mean(  cs), np.std(  cs)

In [ ]:
dfnamesubj2res_Died_fboffl.keys()

In [ ]:
fnf = pjoin(path_data_cc,'dfnamesubj2res_Died.npz')
dfnamesubj2res_Died = np.load(fnf, allow_pickle=1)
dfnamesubj2res_Died = dfnamesubj2res_Died['arr_0'][()]

In [ ]:
# Died
dfs = []
plot0 = 1
for (dfn,subj),out in dfnamesubj2res_Died.items():
    if dfn == 'NIH':
        dfos = dfcc_all_NIH.query('subject == @subj')
        coln_error_ = 'error'
        error   = dfos[coln_error_].to_numpy() * 180 / np.pi
        trial_index = dfos['trials']
        dfcur = dfcc_all_NIH        
    else:
        dfos = dfcc_all.query('subject == @subj  and trial_index >= @numtrain')
        coln_error_ = coln_error
        error   = dfos[coln_error_].to_numpy() 
        trial_index = dfos['trial_index']
        dfcur = dfcc_all
    #for out in out_cursubj:
#for out in [outT, outD]:
        
    perturb = dfos['perturbation'].to_numpy()    
    #out = subj2out_Tan[subj ]        
    if 'adaptation_rate' in out:
        adaptationRate = out['adaptation_rate']
        output = out['y_pred']
                
        adaptationRate2 = out['adaptation_rate']
        output2 = out['y_pred']
    else:
        adaptationRate = None
        out2 = dfnamesubj2res_Died_fboffl[(dfn,subj)]
        output = out['output']
        output2 = out2['output']

    dfcur.loc[dfos.index, 'error_pred_Died'] = -output+perturb
    dfcur.loc[dfos.index, 'state_Died'] = out['states']
    #dfcur.loc[dfos.index, 'err_sens_Tan'] = err_sens2
    
#     df = pd.DataFrame({'error':error, 
#                        'error_pred':output,
#                       'pert':perturb,
#                        'err_sens':adaptationRate,
#                       'trial_index':trial_index})
#     df['subject'] = subj
#     df['dfname'] = dfn
#     df['model'] = 'Diedrichsen'
#     dfs += [df]
    
    if plot0:
        plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
        plt.title('Subject ' + dfos['subject'].values[0]) # Use string concatenation to make titles
        plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
        plt.plot(-output+perturb, 
                 linewidth=2, label='Model prediction',alpha=0.8)
        #plt.plot( -output2+perturb, 
        #         linewidth=2, label='mp_fboffl',alpha=0.8)

        plt.ylabel('Angle (°)')
        plt.xlabel('Trials')
        plt.plot(perturb, label='pert', color='red', ls=':')
        plt.legend() # Call legend function to show legends
        #plt.twinx() # Use twinx function to create a second y-axis
        if adaptationRate is not None:
            plt.plot(30 + 50*adaptationRate, linewidth=1, label='Adaptation rate', color='green')            
            #plt.plot(30 + 50*adaptationRate2, linewidth=1, label='Adaptation rate', color='darkgreen')            
            plt.legend() # Call legend function to show legends
        plt.ylim(-40,60)
        plt.grid(True)
        plt.show() # Call show function to display the figure
    #break
    

In [ ]:
np.max(output)

In [ ]:
np.std(output - output2) / np.mean(np.abs(output) )

In [ ]:
dfr_D=  pd.concat(dfs).reset_index()

In [ ]:
len(output), len(error)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
numtrain = 12

In [ ]:
# Tan
#dfs = []
plot0 = 1
for (dfn,subj),out in dfnamesubj2res_Tan.items():
    if dfn == 'NIH':
        dfcur = dfcc_all_NIH
        dfos = dfcc_all_NIH.query('subject == @subj')
        coln_error_ = 'error'
        error   = dfos[coln_error_].to_numpy() * 180 / np.pi
        trial_index = dfos['trials']
    else:
        dfcur = dfcc_all
        dfos = dfcc_all.query('subject == @subj  and trial_index >= @numtrain')
        coln_error_ = coln_error
        error   = dfos[coln_error_].to_numpy() 
        trial_index = dfos['trial_index']
        
        #break
    #for out in out_cursubj:
#for out in [outT, outD]:
    #error   = dfos[coln_error_].to_numpy() * 180 / np.pi
    perturb = dfos['perturbation'].to_numpy()    
    #out = subj2out_Tan[subj ]
    adaptationRate = None
    if 'adaptation_rate' in out:
        adaptationRate = out['adaptation_rate']
    output = out['y_pred']
    
    dfcur.loc[dfos.index, 'error_pred_Tan'] = output
    dfcur.loc[dfos.index, 'err_sens_Tan'] = adaptationRate
    dfcur.loc[dfos.index, 'state_Tan'] = out['states']
    
#     df = pd.DataFrame({'error':error, 
#                        'error_pred':output,
#                       'pert':perturb,
#                        'err_sens':adaptationRate,
#                       'trial_index':trial_index})
#     df['subject'] = subj
#     df['dfname'] = dfn
#     df['model'] = 'Tan'
#     dfs += [df]
    Net, coef = getTanModelNte(subj)
    if plot0:
        plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
        plt.title('Subject ' + dfos['subject'].values[0] + f' Net={Net}, c={coef:.4f}') # Use string concatenation to make titles
        plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
        plt.plot(output, linewidth=2, label='Model prediction',alpha=0.8)
        plt.ylabel('Angle (°)')
        plt.xlabel('Trials')
        plt.plot(perturb, label='pert', color='red', ls=':')
        plt.legend() # Call legend function to show legends
        #plt.twinx() # Use twinx function to create a second y-axis
        if adaptationRate is not None:
            plt.plot(30 + 50*adaptationRate, linewidth=1, label='Adaptation rate', color='green')            
            plt.legend() # Call legend function to show legends
        plt.ylim(-40,60)
        plt.grid(True)
        plt.show() # Call show function to display the figure
    #break
#dfr_T=  pd.concat(dfs).reset_index()

In [ ]:
len(output), len(dfos)

In [ ]:
#dfr_T0 = dfr_T

In [ ]:
from taumodels.models import calcH_orig

In [ ]:
len(err_sens2), len(output), len(error)

In [ ]:
cols = ['trialwb',
 'trialwe',
 'trialwpertstage_wb',
 'trialwpert_wb',
 'trialwpert_we',
 'trialwtgt',
 'trialwtgt_wpert_wb',
 'trialwtgt_wpertstage_wb',
 'trialwtgt_wpertstage_we',
 'trialwtgt_wpert_we',
 'trialwtgt_we',
 'trialwtgt_wb',
 'trialwpert',
 'trialwtgt_wpert'] + ['movement_duration','RT']
','.join(cols)

In [ ]:
[coln for coln in dfcc_all_NIH.columns if coln.find('RT') >= 0]

In [ ]:
fnf = pjoin(path_data_cc,'dfnamesubj2res_Herz.npz')
dfnamesubj2res_Herz = np.load(fnf, allow_pickle=1)
dfnamesubj2res_Herz = dfnamesubj2res_Herz['arr_0'][()]

In [ ]:
from taumodels.models import getBestHerzPar, calcH_orig

In [ ]:
out.keys()

In [ ]:
# Herz
dfs = []
plot0 = 1
for (dfn,subj),out in dfnamesubj2res_Herz.items():
    if dfn == 'NIH':
        dfcur = dfcc_all_NIH
        dfos = dfcur.query('subject == @subj')
        coln_error_ = 'error'
        error   = dfos[coln_error_].to_numpy() * 180 / np.pi
        trial_index = dfos['trials'].values
        perturb = dfos['perturbation'].values
        EC_mask = np.zeros_like(perturb)
    else:
        dfcur = dfcc_all
        dfos = dfcur.query('subject == @subj  and trial_index >= @numtrain')
        coln_error_ = coln_error
        error   = dfos[coln_error_].to_numpy() 
        trial_index = dfos['trial_index']
        perturb = dfos['perturbation'].values
        EC_mask = (dfos['trial_type'] == "error_clamp").values
        
    best_par = getBestHerzPar(out)
    if best_par is None:
        print('not par found for ',dfn,subj)
        continue
    
    pard = best_par.copy()
    for k in 'fun,nit,nfev,seed'.split(','):
        del pard[k]
    r = calcH_orig(-perturb,EC_mask=EC_mask, **pard)
    (motor_output,error_pred2,ws,err_sens2, 
        gaussian_centers,gaussian_variances) = r 
    output = error_pred2
    #np.array(ws).shape == (767,20)
    
    dfcur.loc[dfos.index, 'error_pred_Herz'] = output
    dfcur.loc[dfos.index, 'err_sens_Herz'] = err_sens2
    # there is no "state" in Herzfeld, only motor output
    dfcur.loc[dfos.index, 'state_Herz'] = motor_output
    
#     df = pd.DataFrame({'error':error, 
#                    'error_pred':output,
#                   'pert':perturb,
#                    'err_sens':err_sens2,
#                   'trial_index':trial_index})
#     df['subject'] = subj
#     df['dfname'] = dfn
#     df['model'] = 'Herzfeld_mod'
#     df['error_diff'] = df['error'] - df['error_pred']
#     dfs += [df]
    
    
    if plot0:
        plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
        plt.title('Subject ' + dfos['subject'].values[0]) # Use string concatenation to make titles
        plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
        plt.plot(output, linewidth=2, label='Model prediction',alpha=0.8)
        plt.plot(motor_output, linewidth=2, label='state')
        plt.ylabel('Angle (°)')
        plt.xlabel('Trials')
        plt.plot(perturb, label='pert', color='red', ls=':')
        plt.legend() # Call legend function to show legends
        #plt.twinx() # Use twinx function to create a second y-axis
        if adaptationRate is not None:
            plt.plot(30 + 50*adaptationRate, linewidth=1, label='Adaptation rate', color='green')            
            plt.legend() # Call legend function to show legends
        plt.ylim(-40,60)
        plt.grid(True)
        plt.show() # Call show function to display the figure
    #break

#dfr_H=  pd.concat(dfs).reset_index()

In [ ]:
#np.quantile?

In [ ]:
cols = 'trials,error_pred_Tan,error_pred_Herz,error_pred_Died,err_sens_Herz,err_sens_Tan,prev_error,error'.split(',')
for col in cols:
    if col not in dfcc_all_NIH:
        print(col)

In [ ]:
import seaborn as sns

In [ ]:
#dfr = pd.concat([dfr_D, dfr_T, dfr_H])

In [ ]:
for vn in ['movement','belief', 'state_Died','state_Tan','state_Herz']:
    vneff = vn
    if vn == 'movement':
        vneff = 'org_feedback'
    dfcc_all_NIH[f'prev_{vn}'] = dfcc_all_NIH.groupby('subject')[vneff].shift(1)

print(path_data)
from behav_proc import pscAdj_NIH, reshiftPi
cols = ['error_pred_Tan', 'error_pred_Died', 'error_pred_Herz']
pscAdj_NIH(dfcc_all_NIH, cols)

cols = ['state_Tan', 'state_Died', 'state_Herz']
pscAdj_NIH(dfcc_all_NIH, cols)

#dfcc_all_NIH['error_deg'] = dfcc_all_NIH['error'] * 180 / np.pi
reshiftPi(dfcc_all_NIH,'error')
dfcc_all_NIH['error_deg'] = dfcc_all_NIH['error'] / np.pi * 180 

def f(x):    
    PI = 180
    if x > PI:
        x -= 2*PI
    elif x < -PI:
        x += 2*PI
    return x
dfcc_all_NIH['error_pred_Tan']  = dfcc_all_NIH['error_pred_Tan'].apply(f)
dfcc_all_NIH['error_pred_Died'] = dfcc_all_NIH['error_pred_Died'].apply(f)
dfcc_all_NIH['error_pred_Herz'] = dfcc_all_NIH['error_pred_Herz'].apply(f)

from config2 import envcode2env
dfcc_all_NIH['env'] = dfcc_all_NIH['environment'].apply(lambda x: envcode2env[x])


pscAdj_NIH(dfcc_all_NIH, ['error_deg'])
dfcc_all_NIH['error_diff_Tan']  = dfcc_all_NIH['error_deg'] - dfcc_all_NIH['error_pred_Tan']
dfcc_all_NIH['error_diff_Died'] = dfcc_all_NIH['error_deg'] - dfcc_all_NIH['error_pred_Died']
dfcc_all_NIH['error_diff_Herz'] = dfcc_all_NIH['error_deg'] - dfcc_all_NIH['error_pred_Herz']
dfcc_all_NIH['error_diff_Tan']  = dfcc_all_NIH['error_diff_Tan'].apply(f)
dfcc_all_NIH['error_diff_Died'] = dfcc_all_NIH['error_diff_Died'].apply(f)
dfcc_all_NIH['error_diff_Herz'] = dfcc_all_NIH['error_diff_Herz'].apply(f)

dfcc_all_NIH['error_abs'] = dfcc_all_NIH['error'].abs()
dfcc_all_NIH['prev_error_abs'] = dfcc_all_NIH['prev_error'].abs()

pscAdj_NIH(dfcc_all_NIH, 'error_diff_Tan,error_diff_Died,error_diff_Herz'.split(','))
fn = 'row_frame_wmodel_output.pkl.zip'
dfcc_all_NIH.to_pickle(pjoin(path_data, fn), compression='zip')

In [ ]:
from shutil import copyfile 
r = copyfile( pjoin(path_data, fn), '/home/demitau/ju_lyon/memerr/data2/' + fn )
print(r)
r = copyfile( pjoin(path_data, fn), pjoin('/home/demitau/ownCloud/Current/merr_data', fn ) )
print(r)

In [ ]:
dfcc_all_NIH['state_Died']

In [ ]:
dfcc_all_NIH['error_diff_Tan'].abs().max()
dfcc_all_NIH['error_diff_Died'].abs().max()
dfcc_all_NIH['error_diff_Herz'].abs().max()

In [ ]:
sns.set(font_scale=2)

In [ ]:
dfes_ = pd.read_pickle('/home/demitau/data_Quentin/full_experiments/data2/df_all_multi_tsz.pkl.zip')

dfes = dfes_.query('trial_group_col_calc == "trials" and trial_shift_size == 1').copy()

assert not dfes.duplicated(['subject','trials']).any()
print(dfes.groupby('subject')['trials'].max().std())

dfes['err_sens_clip'] = dfes['err_sens'].clip(-1,1)
dfes['err_sens_clip3'] = dfes['err_sens'].clip(-3,3)

In [ ]:
pertplot = {'color': 'r', 'alpha': 0.9, 'ls': ':'}

In [ ]:
from behav_proc import truncateDf
dfes_trunc_by1 = truncateDf(dfes,'err_sens',inplace=False,
                       q=None, low=-1,hi=1,
                       infnan_handling='discard')

dfes_trunc_q = truncateDf(dfes,'err_sens',inplace=False,
                       q=0.05,
                       infnan_handling='discard')

In [ ]:
dfes_trunc_by3 = truncateDf(dfes,'err_sens',inplace=False,
                       q=None, low=-3,hi=3,
                       infnan_handling='discard')


In [ ]:
len(dfes_trunc) / len(dfes),len(dfes_trunc_by3) / len(dfes), len(dfes_trunc_q)/len(dfes)

In [ ]:
dfc_p = dfcc_all_NIH.query(f'subject == "{subjects_NIH[0]}"')
dfc_p = dfc_p.sort_values('trials')
pert = dfc_p['perturbation'].values[:192*4]
tr   = dfc_p['trials'].values[:192*4]
envv = dfc_p['environment'].values[:192*4].astype(float)
envv[envv == 0] = np.nan
pert[envv == 1] = np.nan

In [ ]:
fg = sns.relplot(data=dfes, 
            kind='line',x='trials',
            y ='err_sens', aspect=3, facet_kws={'sharex': 'col'},
                ci='sd')
ax = fg.axes.flatten()[0]
_c = 0.5
ax.plot(pert / 30 * _c + _c, **pertplot)
#ax.set_title('H. Tan model')
ax.axhline(0,c='r',ls=':')
ax.set_ylabel('Error sensitivity')
ax.set_title('no trunc')
ax.grid(True)

fg = sns.relplot(data=dfes_trunc_by1, 
            kind='line',x='trials',
            y ='err_sens', aspect=3, facet_kws={'sharex': 'col'},
                ci='sd')
ax = fg.axes.flatten()[0]
_c = 0.5
ax.plot(pert / 30 * _c + _c, **pertplot)
#ax.set_title('H. Tan model')
ax.axhline(0,c='r',ls=':')
ax.set_ylabel('Error sensitivity')
ax.set_title('trunc when abs > 1')
ax.grid(True)

fg = sns.relplot(data=dfes_trunc_by3, 
            kind='line',x='trials',
            y ='err_sens', aspect=3, facet_kws={'sharex': 'col'},
                ci='sd')
ax = fg.axes.flatten()[0]
_c = 0.5
ax.plot(pert / 30 * _c + _c, **pertplot)
#ax.set_title('H. Tan model')
ax.axhline(0,c='r',ls=':')
ax.set_ylabel('Error sensitivity')
ax.set_title('trunc when abs > 3')
ax.grid(True)

fg = sns.relplot(data=dfes_trunc_q, 
            kind='line',x='trials',
            y ='err_sens', aspect=3, facet_kws={'sharex': 'col'},
                ci='sd')
ax = fg.axes.flatten()[0]
_c = 0.5
ax.plot(pert / 30 * _c + _c, **pertplot)
#ax.set_title('H. Tan model')
ax.axhline(0,c='r',ls=':')
ax.set_ylabel('Error sensitivity')
ax.set_title('trunc when q > 0.05')
ax.grid(True)

In [ ]:
from behav_proc import correctPertCol_NIH
subj = subjects_NIH[0]
correctPertCol_NIH(dfes)

In [ ]:
pert

In [ ]:
dfcc_all_NIH['err']

In [ ]:
#err sens
sns.set_style('whitegrid')
# df_ = dfes.query('subject == @subj')
# print ( len(df_) )
# fg = sns.relplot(data=df_, 
#     kind='line',x='trials',
#     y ='perturbation', aspect=3, facet_kws={'sharex': 'col'},
#         ci='sd')
# ax = fg.axes.flatten()[0]
# ax.set_title('Perturbation schedule')
# ax.set_ylabel('perturbation [deg]')
# ax.set_ylim(-35,35)
# ax.set_xlabel('')
# ax.grid(True)
tpls = zip( ['err_sens_clip','err_sens_Tan','err_sens_Herz'], ['Empirical data (clipped to [-1,1])',
                                                       'H. Tan model','D. Herzfeld model'],
          [dfes, dfcc_all_NIH, dfcc_all_NIH])

ww=17
fig,axs = plt.subplots(3,1,figsize=(ww,3*ww/3.5))
for axi,(vn,vstr,df_) in enumerate(tpls):
    ax = axs[axi]
    sns.set(font_scale=2.5)
    with warnings.catch_warnings() as w:
        warnings.filterwarnings('ignore',category = FutureWarning)
        fg = sns.lineplot(data=df_, x='trials',
                y =vn, ci='sd', ax=ax)
    ax.axhline(0,c='r',ls=':')
    ax.set_title(vstr)
    ax.set_ylabel('Error sensitivity\nmean over\nsubjects')
    ax.set_ylim(-1,1.3)
    ax.set_xlabel('')
    ax.grid(True)
    N = 40; pertc = 1/N
    vshift = 0; 
    ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label=f'perturbation / {N}')
    #ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
    #                   alpha=0.2, label='environment')
    ax.set_xticklabels([])
    ax.set_xlabel('')

# fg = sns.relplot(data=dfcc_all_NIH, 
#             kind='line',x='trial_index',
#             y ='err_sens_Tan', aspect=3, facet_kws={'sharex': 'col'},
#                 ci='sd')
# ax = fg.axes.flatten()[0]
# ax.set_title('H. Tan model')
# ax.axhline(0,c='r',ls=':')
# ax.set_ylabel('Error sensitivity')
# ax.set_xlabel('')
# ax.set_ylim(-1,1.3)
# ax.grid(True)
# ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
# ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
#                    alpha=0.2, label='environment')
# ax.set_xticklabels([])
# ax.set_xlabel('')

# fg = sns.relplot(data=dfcc_all_NIH, 
#             kind='line',x='trial_index',
#             y ='err_sens_Herz', aspect=3, facet_kws={'sharex': 'col'},
#                 ci='sd')
# ax = fg.axes.flatten()[0]
# ax.set_title('D. Herzfeld model')
# ax.axhline(0,c='r',ls=':')
# ax.set_ylabel('Error sensitivity')
# ax.set_ylim(-1,1.3)
# ax.set_xlabel('Trial index')
# ax.grid(True)
# ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
# ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
#                    alpha=0.2, label='environment')
ax.legend(loc='lower right', markerscale=4.)

# Create two line objects
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
blue_line = Line2D([], [], color='tab:blue', alpha=1, lw=4)
pink_dashed_line = Line2D([], [], color='violet', linestyle='dashed', lw=4)
blue_rectangle = mpatches.Rectangle((0, 0), 1, 1, color='tab:blue', alpha=0.3)

# Add the line objects to the legend
ax.legend(handles=[blue_line, pink_dashed_line,blue_rectangle], 
          labels=['state', f'perturbation / {N}','ES $\sigma$'], 
          loc='lower right', frameon=True)

        
from config2 import path_fig
plt.tight_layout()
plt.savefig(pjoin(path_fig,'model_ES_dyn.pdf'))
plt.savefig(pjoin(path_fig,'model_ES_dyn.svg'))

In [ ]:
varn2pub1={'error_deg':'Obeserved error', 
          'error_pred_Herz':'Herzfeld error prediction',
         'error_pred_Tan':'Tan error prediction',
         'error_pred_Died':'Diedrichsen error prediction'}
varn2pub2={'err_sens':'Empirical ES', 
          'err_sens_Herz':'Herzfeld ES',
         'err_sens_Tan':'Tan ES'}

In [ ]:
# errr pred pscadj
cols=['error_deg', 'error_pred_Died','error_pred_Tan',
          'error_pred_Herz']
for ci,coln in enumerate(cols):
    fg = sns.relplot(data=dfcc_all_NIH, 
                kind='line',x='trial_index',
                y =coln + '_pscadj', aspect=3, facet_kws={'sharex': 'col'},
                    ci='sd',height=5)
    ax = fg.axes.flatten()[0]
    ax.set_title(varn2pub1[coln])
    ax.set_ylabel('error [deg]')
    ax.axhline(0,c='r',ls=':')
    ax.grid(True,which='both')
    ax.set_ylim(-50,50)
    
    pertc = 1.
    ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
    ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
                   alpha=0.2, label='environment')
    ax.set_xlabel('Trial index')
    if ci != (len(cols) - 1):
        ax.set_xticklabels([])
        ax.set_xlabel('')

In [ ]:
# orig, pred, orig - pred for one model 
for coln in ['error_deg_pscadj', 'error_pred_Tan_pscadj', 'error_diff_Tan_pscadj']:
    fg = sns.relplot(data=dfcc_all_NIH, 
            kind='line',x='trial_index',
            y =coln, aspect=3, facet_kws={'sharex': 'col'},
                ci='sd')
    ax = fg.axes.flatten()[0]
    ax.set_title(coln)
    ax.axhline(0,c='r',ls=':')

In [ ]:
for coln in [coln + '_pscadj' for coln in 'error_diff_Tan,error_diff_Died,error_diff_Herz'.split(',')]:
    fg = sns.relplot(data=dfcc_all_NIH, 
            kind='line',x='trial_index',
            y =coln, aspect=3, facet_kws={'sharex': 'col'},
                ci='sd')
    ax = fg.axes.flatten()[0]
    ax.set_title(coln)
    ax.axhline(0,c='r',ls=':')
    ax.set_ylim(-30,30)
    ax.grid(True)

In [ ]:
[c for c in dfcc_all_NIH.columns if c.find('state') >= 0]

In [ ]:
dfcc_all_NIH['state_Herz_neg'] = -dfcc_all_NIH['state_Herz']

In [ ]:
cols=[ 'state_Died','state_Tan',
          'state_Herz_neg', 'belief']
cols_ = ['Diedrichsen state','Tan state','Herzfeld state','Empirical state = movement - target']
varn2pub3 = dict(zip(cols,cols_))

In [ ]:
dfcc_all_NIH = pscAdj_NIH(dfcc_all_NIH, ['perturbation'])

In [ ]:
#dfcc_all_NIH['belief']

In [ ]:
dfcc_all_NIH['belief_pscadj' ] = dfcc_all_NIH['perturbation_pscadj'] - dfcc_all_NIH['error_deg_pscadj']

In [ ]:
dfcc_all_NIH['state_Herz_neg_pscadj'] = -dfcc_all_NIH['state_Herz_pscadj']

In [ ]:
# errr pred pscadj
import warnings
sns.set_style('whitegrid')
cols=[ 'belief','state_Died','state_Tan',
          'state_Herz_neg']
ww=17
fig,axs = plt.subplots(4,1,figsize=(ww,4*ww/3.5))
for ci,coln in enumerate(cols):
    ax = axs[ci]
    with warnings.catch_warnings() as w:
        warnings.filterwarnings( 'ignore', category=FutureWarning)
        fg = sns.lineplot(data=dfcc_all_NIH, 
                x='trial_index', y =coln + '_pscadj',  
                    errorbar='sd', ax=ax)
    #ax = fg.axes#.flatten()[0]
    ax.set_title(varn2pub3[coln])
    #ax.set_title(coln)
    ax.set_ylabel('Latent state, mean\nover subjects')
    ax.axhline(0,c='r',ls=':')
    ax.grid(True,which='both')
    ax.set_ylim(-35,35)
    
    pertc = 1.
    ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
    #ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
    #               alpha=0.2, label='environment')
    ax.set_xlabel('Trial index')
    if ci != (len(cols) - 1):
        ax.set_xticklabels([])
        ax.set_xlabel('')
        
# Create two line objects
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
blue_line = Line2D([], [], color='tab:blue', alpha=1, lw=4)
pink_dashed_line = Line2D([], [], color='violet', linestyle='dashed', lw=4)
blue_rectangle = mpatches.Rectangle((0, 0), 1, 1, color='tab:blue', alpha=0.3)

# Add the line objects to the legend
ax.legend(handles=[blue_line, pink_dashed_line,blue_rectangle], 
          labels=['state', 'perturbation','state $\sigma$'], 
          loc='lower right', frameon=True)

        
from config2 import path_fig
plt.tight_layout()
plt.savefig(pjoin(path_fig,'model_state_dyn.pdf'))
plt.savefig(pjoin(path_fig,'model_state_dyn.svg'))

# stats

In [ ]:
colns_errdif = [coln + '_pscadj_abs' for coln in 'error_diff_Died,error_diff_Tan,error_diff_Herz'.split(',')]
dfme = dfcc_all_NIH.groupby(['subject','env'])[colns_errdif].mean().reset_index()
tpls = zip(colns_errdif, ['Mismatch with\nDiedrichsen','Mismatch with\nTan','Mismatch with\nHerzfeld'])
#sns.set_style()
sns.set(style='whitegrid', font_scale=1.5)
fig,axs = plt.subplots(1,4,figsize=(12,4))

for colni,(coln,coln_nice) in enumerate(tpls):
    with warnings.catch_warnings():
        ax = axs[colni]
        warnings.filterwarnings('ignore',category=FutureWarning)
        #plt.figure()
        sns.boxplot(data=dfme, x='env', y = coln, ax = ax, 
                    order = ['stable','random'], 
                   palette=['tab:orange','tab:grey'])#, kind='box')
        ax.set_title(coln_nice)
        ax.set_ylabel('')
        ax.set_ylim(0,17)
        ax.axhline(y=0,ls=':',c='red',lw=2)        
        if colni != 0:
            ax.set_yticklabels([])
        if colni == 0:
            ax.set_ylabel('abs(Error mismatch) [deg]')
        ax.yaxis.grid(True)
        ax.set_xlabel('')
    #break
#fig.subplots_adjust(right=0.8)    
#blue_line = Line2D([], [], color='tab:blue', alpha=1, lw=4)
#pink_dashed_line = Line2D([], [], color='tab:orange', linestyle='dashed', lw=4)
blue_rectangle   = mpatches.Rectangle((0, 0), 1, 1, color='tab:orange')
orange_rectangle = mpatches.Rectangle((0, 0), 1, 1, color='tab:grey')

# Add the line objects to the legend
ax = axs[-1]; ax.set_yticks([])
ax.axis('off')
ax.legend(handles=[blue_rectangle, orange_rectangle], 
          labels=['stable','random'], 
          loc='lower right', frameon=True)    

plt.tight_layout()
plt.savefig(pjoin(path_fig,'model_error_mismatch.pdf'))
plt.savefig(pjoin(path_fig,'model_error_mismatch.svg'))
#plt.tight_layout()
#plt.suptitle('Error mismatch', )

In [ ]:
dfcc_all_NIH['error_diff_Died_pscadj_abs']

In [ ]:
for coln in [coln + '_pscadj' for coln in 'error_diff_Tan,error_diff_Died,error_diff_Herz'.split(',')]:
    dfcc_all_NIH[coln + '_abs'] = dfcc_all_NIH[coln].abs()


In [ ]:
dfrs = []
for coln in [coln + '_pscadj' for coln in 'error_diff_Tan,error_diff_Died,error_diff_Herz'.split(',')]:
    colneff = coln + '_abs'
    r = dfcc_all_NIH.groupby(['subject','env'])[colneff].mean()
    r = r.to_frame().reset_index()
    #r.drop_columns
    r['varname'] = colneff
    r['model'] = coln.split('_')[2]
    r = r.rename(columns={colneff:'valme'})
    #display(r)
    dfrs += [r]
dfrs = pd.concat(dfrs, ignore_index = 1)
dfrs

In [ ]:
from behav_proc import comparePairs

In [ ]:
import pingouin

In [ ]:
ttrssig,ttrs = comparePairs(dfrs.query('env == "random"'), 'valme', 'model', paired=True)

ttrssig.query('alternative != "two-sided" and pooled == True')[['ttstr','pval']]

In [ ]:
ttrssig,ttrs = comparePairs(dfrs.query('env == "stable"'), 'valme', 'model', paired=True)

ttrssig.query('alternative != "two-sided" and pooled == True')[['ttstr','pval']]

In [ ]:
for model in ['Died','Tan','Herz']:
    ttrssig,ttrs = comparePairs(dfrs.query('model == @model'), 'valme', 'env', paired=True)
    r = ttrssig.query('alternative != "two-sided" and pooled == True')[['ttstr','pval']]
    display(model, r)

In [ ]:
dfrs.groupby(['model','env']).mean()

In [ ]:
sns.set_style('white')
#sns.set_style('dark')

In [ ]:
dfr['error_diff'] = dfr['error'] - dfr['error_pred']
dfr['error_diff_abs'] = dfr['error_diff'].abs()

In [ ]:
dfr['error_abs'] = dfr['error'].abs()
dfr['error_pred_abs'] = dfr['error_pred'].abs()

In [ ]:
pert_plot = dfcc_all_NIH.query('subject == @subjects_NIH[0]')['perturbation'].values

In [ ]:
pertplot=dict(color='r', alpha=0.6, ls=':')

In [ ]:
#pertplot=dict(color='r', alpha=1, ls=':')

#plt.figure(figsize=(12,4))
fg = sns.relplot(data=dfr.query('model != "Diedrichsen"'), 
            kind='line',x='trial_index',
            y ='err_sens', col='dfname', row='model',
                aspect=3, facet_kws={'sharex': 'col'},
                ci='sd')
for axi,ax in enumerate(fg.axes.flatten()):
    if axi % 2 ==0:
        #ax.plot(pert_plot, **pertplot)
        ax.plot(0.8 + pert_plot / (30*4), **pertplot)
    ax.axhline(0,c='k',ls=':')
    ax.grid(True)
    

In [ ]:
sns.lineplot?

In [ ]:
#%debug
fg = sns.relplot(data=dfr, kind='line',x='trial_index',
            y ='error_pred_abs', col='dfname', row='model',
                ci='sd')
for axi,ax in enumerate(fg.axes.flatten()):
    if axi % 2 ==0:
        ax.plot(pert_plot, **pertplot)
    ax.axhline(0,c='k',ls=':')
    ax.grid(True)

# fg = sns.relplot(data=dfr, kind='line',x='trial_index',
#             y ='error_pred_abs', col='dfname', row='model')
# for ax in fg.axes.flatten():
#     ax.plot(perturb , **pertplot)

In [ ]:
#%debug
fg = sns.relplot(data=dfr, kind='line',x='trial_index',
            y ='error_diff', col='dfname', row='model',
                ci='sd')
for axi,ax in enumerate(fg.axes.flatten()):
    if axi % 2 ==0:
        ax.plot(pert_plot, **pertplot)
    ax.axhline(0,c='k',ls=':')
    ax.grid(True)

# fg = sns.relplot(data=dfr, kind='line',x='trial_index',
#             y ='error_pred_abs', col='dfname', row='model')
# for ax in fg.axes.flatten():
#     ax.plot(perturb , **pertplot)

In [ ]:
fg.axes.shape

In [ ]:
fg = sns.relplot(data=dfr, kind='line',x='trial_index',
            y ='error_diff_abs', col='dfname', row='model',
                 ci='sd')
for axi,ax in enumerate(fg.axes.flatten()):
    if axi % 2 ==0:
        ax.plot(pert_plot, **pertplot)
    ax.axhline(0,c='k',ls=':')
    ax.grid(True)

In [ ]:
fg = sns.relplot(data=dfr, kind='line',x='trial_index',
            y ='error_abs', col='dfname', row='model')
for ax in fg.axes.flatten():
    ax.plot(perturb, **pertplot)

fg = sns.relplot(data=dfr, kind='line',x='trial_index',
            y ='error_pred_abs', col='dfname', row='model')
for ax in fg.axes.flatten():
    ax.plot(perturb , **pertplot)